<a href="https://colab.research.google.com/github/saito-structural-data/hotel-supply-demand-etl/blob/main/hotel_demand_supply_recovery_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ライブラリのインポート

In [8]:
import os
import re
import glob
import unicodedata
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler
from google.colab import drive
import warnings

# Setting warnings to ignore for cleaner output, especially for chained assignments and runtime warnings.
warnings.filterwarnings('ignore')

# データ前処理・整形ヘルパー関数

In [9]:
# ==============================================================================
# 需要データ前処理・整形ヘルパー関数
# ==============================================================================

def parse_year(cell):
    """和暦から西暦を返す"""
    if not isinstance(cell, str):
        return None

    if "平成" in cell:
        num_str = re.sub(r"\D", "", cell)
        if not num_str:
            return None
        return 1988 + int(num_str)

    if "令和" in cell:
        num_str = re.sub(r"\D", "", cell)
        if not num_str:
            return None
        return 2018 + int(num_str)

    # 4桁西暦の可能性も考慮
    if re.match(r"^\d{4}$", cell.strip()):
        return int(cell.strip())

def wide_pref_to_long(df_wide: pd.DataFrame, value_col_name: str) -> pd.DataFrame:
    """宿泊統計の都道府県×月の横持ち表を縦持ちに変換する"""

    # --- 年・月行を取り出し ---
    year_row = df_wide.iloc[0]
    month_row = df_wide.iloc[1]

    # --- 年月対応表を作成 ---
    current_year = None
    cols = []
    for col in year_row.index:
        y = parse_year(year_row[col])
        if y is not None:
            current_year = y

        cell = month_row[col]
        # 月を含むセルと、有効な年が揃っている列を抽出
        if isinstance(cell, str) and "月" in cell and current_year is not None:
            # セル内容が NaN の場合に備え、floatからintへの変換時にエラーが出ないよう防御
            try:
                month = int(cell.replace("月", "").replace(" ", "").replace("　", ""))
                if 1 <= month <= 12:
                     cols.append((col, current_year, month))
            except ValueError:
                continue
        # 連続する列で月データがない場合も、前の年データが継続しているとみなすため、current_yearはリセットしない

    # --- 本体データ（都道府県行）を整形 ---
    data = df_wide.rename(columns={0: "pref"})
    data["pref"] = data["pref"].astype(str)

    # 全国・その他の行を除外 (都道府県コードでフィルタリング)
    # A列が '01北海道', '47沖縄県' の形式を想定
    data = data[data["pref"].str.match(r"^\d{2}", na=False)]

    # コード & 名称に分割
    data["pref_code"] = data["pref"].str[:2].astype(int)
    data["pref_name"] = data["pref"].str[2:]

    # --- 横→縦に展開 ---
    long_rows = []
    for col, year, month in cols:
        tmp = data[["pref_code", "pref_name", col]].copy()
        tmp = tmp.rename(columns={col: value_col_name})
        tmp["year"] = year
        tmp["month"] = int(month)
        long_rows.append(tmp)

    if not long_rows:
        return pd.DataFrame()

    df_long = pd.concat(long_rows, ignore_index=True)

    # --- 日付列を付与 ---
    df_long["date"] = pd.to_datetime(
        dict(year=df_long["year"], month=df_long["month"], day=1)
    )
    df_long["yyyymm"] = df_long["date"].dt.strftime("%Y-%m")

    # --- 列の並びを整理 ---
    df_long = df_long[[
        "date", "yyyymm", "pref_code", "pref_name",
        "year", "month", value_col_name
    ]].sort_values(["pref_code", "date"]).reset_index(drop=True)

    return df_long

def merge_hotel_stats(
    total_df: pd.DataFrame,
    jap_df: pd.DataFrame,
    foreign_df: pd.DataFrame,
    occ_df: pd.DataFrame
) -> pd.DataFrame:
    """4つの宿泊統計（long形式）をマージしてマスターデータを作成する"""

    # --- 結合キーを統一 ---
    keys = ["pref_code", "yyyymm"]

    # --- マージ開始 ---
    df = total_df.merge(jap_df[keys + ["japanese_guest"]],
                        on=keys, how="left")

    df = df.merge(foreign_df[keys + ["foreigner_guest"]],
                  on=keys, how="left")

    df = df.merge(occ_df[keys + ["occ"]],
                  on=keys, how="left")

    # --- データ型チェック（必要に応じて） ---
    for col in ["occ", "total_guest", "japanese_guest", "foreigner_guest"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # --- 日付をつける（統一） ---
    df["date"] = pd.to_datetime(df["yyyymm"] + "-01")

    # --- 並び順 ---
    df = df.sort_values(["pref_code", "date"]).reset_index(drop=True)

    return df

# ==============================================================================
# 供給データ前処理・整形ヘルパー関数
# ==============================================================================

def normalize(s):
    return unicodedata.normalize("NFKC", s)

def find_first_table_sheet(sheet_names):
    """シート名から『第1表』『1月』を含む *参考ではない* ものを探す"""
    normalized = [normalize(x) for x in sheet_names]

    # 1) 「参考」を含まない 第1表 + 1月（最優先）
    for raw, norm in zip(sheet_names, normalized):
        if ("第1表" in norm) and ("1月" in norm) and ("参考" not in norm):
            return raw

    # 2) それでも見つからなければ適当に先頭
    if sheet_names:
        return sheet_names[0]
    return None

def get_year_from_file(path):
    """A7セルから年を取得。失敗したら None"""
    try:
        xls = pd.ExcelFile(path)
    except Exception:
        return None

    sheet = find_first_table_sheet(xls.sheet_names)
    if sheet is None:
        return None

    try:
        df = pd.read_excel(xls, sheet_name=sheet, header=None)
        # 6行目(0-index) 0列目 (A7セル)
        year_cell = str(df.iloc[6, 0])
        return parse_jp_year(year_cell)
    except Exception:
        return None

def extract_year_from_filename(path: str) -> int | None:
    """
    '2015_facility.xlsx' などのフルパス or ファイル名から 2015 を返す。
    見つからなければ None。
    """
    base = os.path.basename(path)
    m = re.search(r"(20\d{2})", base)
    if m:
        return int(m.group(1))
    return None

def rename_files_by_year(paths, suffix="_facility", dry_run=True):
    for path in paths:
        base = os.path.basename(path)
        ext = os.path.splitext(base)[1]

        # 1) シートから読む
        year = get_year_from_file(path)

        # 2) 読めなければファイル名で推定
        if year is None:
            year = extract_year_from_filename(base)

        if year is None:
            print(f"[SKIP] 年が読めない: {base}")
            continue

        new_name = f"{year}{suffix}{ext}"
        new_path = os.path.join(os.path.dirname(path), new_name)

        if dry_run:
            print(f"[DRY-RUN] {base} -> {new_name}")
        else:
            try:
                os.rename(path, new_path)
                print(f"[RENAMED] {base} -> {new_name}")
            except Exception as e:
                print(f"[ERROR] ファイル名変更失敗 {base}: {e}")

def get_month_from_sheet(sheet_name: str) -> int | None:
    """'第1表（1月）', '第１表(１月)' などから 1 を返す。"""
    name_norm = unicodedata.normalize("NFKC", sheet_name)
    m = re.search(r"(\d+)月", name_norm)
    if m:
        return int(m.group(1))
    return None

def list_first_tables(xls: pd.ExcelFile) -> list[str]:
    """Excel ファイル内の『第1表(○月)』シートだけを返す。『参考第1表(○月)』などは除外する。"""
    result = []
    for s in xls.sheet_names:
        name_norm = unicodedata.normalize("NFKC", s)
        if ("第1表" in name_norm) and ("月" in name_norm) and ("参考" not in name_norm):
            result.append(s)
    return result

def read_sheet_dynamic(xls: pd.ExcelFile, sheet: str) -> pd.DataFrame:
    """
    指定シートを読み込み、
    ・B列で最初に数値が出てくる行を検出してデータ開始とみなす
    ・A列=所在地, B列=総数 だけを返す
    ・都道府県（〜都/道/府/県 で終わる行）だけを残す
    """
    df_raw = xls.parse(sheet_name=sheet, header=None)

    # B列に最初に数値が出る行を探す
    start_row = None
    for i in range(len(df_raw)):
        val = df_raw.iloc[i, 1]
        if isinstance(val, (int, float)) and not pd.isna(val):
            start_row = i
            break

    if start_row is None:
        raise ValueError(f"データ開始行が見つからない: sheet={sheet}")

    # データ部だけ切り出し
    # 0列目=都道府県名, 1列目=施設数総数
    df = df_raw.iloc[start_row:, [0, 1]].copy()
    df.columns = ["pref_name", "facility_total"]

    # 欠損行を削除
    df = df[df["pref_name"].notna() & df["facility_total"].notna()].copy()
    df["pref_name"] = df["pref_name"].astype(str).str.strip()
    df["facility_total"] = pd.to_numeric(df["facility_total"], errors="coerce")

    # 「全国」「◯◯運輸局」などを落として、都道府県だけ残す
    mask_pref = df["pref_name"].str.match(r".+[都道府県]$")
    df = df[mask_pref].reset_index(drop=True)

    # 都道府県名の前に含まれるコードを削除
    df["pref_name"] = (
        df["pref_name"]
        .apply(lambda s: unicodedata.normalize("NFKC", s).strip())
        .str.replace(r"^\d{2}", "", regex=True) # 先頭2桁を削除
        .str.strip()
    )

    return df

def load_facility_year(path: str) -> pd.DataFrame:
    """
    1つの Excel ファイル（1年分）から
    都道府県 × 月 の施設数ロングデータを作る。
    """
    year = extract_year_from_filename(path)
    if year is None:
        raise ValueError(f"ファイル名から年が読めない: {path}")

    xls = pd.ExcelFile(path)
    table_sheets = list_first_tables(xls)

    dfs = []
    for s in table_sheets:
        month = get_month_from_sheet(s)
        if month is None:
            print(f"[SKIP] 月が読めないシート: {s}")
            continue

        try:
            df_m = read_sheet_dynamic(xls, s)
            df_m["year"] = year
            df_m["month"] = month
            dfs.append(df_m)
        except Exception as e:
            print(f"[ERROR] シート処理失敗 {s} in {os.path.basename(path)}: {e}")
            continue

    if not dfs:
        raise ValueError(f"有効な『第1表』シートがない: {path}")

    return pd.concat(dfs, ignore_index=True)

# 需給データETL

In [ ]:
print("--- I. 需要データ ETL ---")
SKIP_ROWS = 2

# ⚠️ファイルパスは実行環境に合わせて適宜修正してください
file_path = "/content/drive/MyDrive/Colab Notebooks/ポートフォリオ/hotel-supply-demand-etl/raw_data/202508_transition_table.xlsx"

# Google Driveがマウントされていない場合はマウント
if not os.path.exists(file_path):
    print("Google Driveをマウントします...")
    try:
        drive.mount('/content/drive')
    except Exception as e:
        print(f"エラー: Google Driveマウント失敗。ファイルパスを確認してください。{e}")

try:
    total_guest = pd.read_excel(file_path, sheet_name="5-2", header=None, skiprows=SKIP_ROWS)
    japanese_guest = pd.read_excel(file_path, sheet_name="6-2", header=None, skiprows=SKIP_ROWS)
    foreigner_guest = pd.read_excel(file_path, sheet_name="7-2", header=None, skiprows=SKIP_ROWS)
    occ = pd.read_excel(file_path, sheet_name="8-2", header=None, skiprows=SKIP_ROWS)

    # データ整形
    total_long = wide_pref_to_long(total_guest, "total_guest")
    japanese_long = wide_pref_to_long(japanese_guest, "japanese_guest")
    foreigner_long = wide_pref_to_long(foreigner_guest, "foreigner_guest")
    occ_long = wide_pref_to_long(occ, "occ")

    # データ結合
    df_master = merge_hotel_stats(
        total_long,
        japanese_long,
        foreigner_long,
        occ_long
    )
    df_master.to_csv("hotel_master.csv", index=False, encoding="utf-8-sig")
    print(df_master.head())
    print("✔ hotel_master.csv を出力しました")

except FileNotFoundError:
    print(f"エラー: 需要データファイルが見つかりません: {file_path}")
    # ダミーデータフレームを作成して、以降の処理を中断
    df_master = None
except Exception as e:
    print(f"エラー: 需要データ処理中に問題が発生しました: {e}")
    df_master = None


--- I. 需要データ ETL ---
Google Driveをマウントします...
Mounted at /content/drive


In [ ]:
print("\n--- II. 供給データ ETL ---")

# ⚠️ファイルパスは実行環境に合わせて適宜修正してください
DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/ポートフォリオ/hotel-supply-demand-etl/processed/001007484(2007)_20251203232706"

# .xls / .xlsx を全部拾う
paths = sorted(glob.glob(os.path.join(DATA_DIR, "*.xls*")))
print(f"{len(paths)} files found in {DATA_DIR}")

# ファイル名変更（ドライランで確認後、本番実行）
# rename_files_by_year(paths, suffix="_facility", dry_run=True)
rename_files_by_year(paths, suffix="_facility", dry_run=False) # 実際に実行する場合はコメントアウトを外す

# ファイル名が変更されたことを前提に、再度ファイルリストを取得
paths = sorted(glob.glob(os.path.join(DATA_DIR, "*_facility.xls*")))


try:
    all_list = []
    for p in paths:
        print("processing:", os.path.basename(p))
        df_year = load_facility_year(p)
        all_list.append(df_year)

    facility_long = pd.concat(all_list, ignore_index=True)

    # 日付列（year+month → 1日）
    facility_long["date"] = pd.to_datetime(
        dict(year=facility_long["year"], month=facility_long["month"], day=1)
    )

    # 都道府県名を正規化し、重複削除・並び替え
    facility_long = (
        facility_long
        .sort_values(["pref_name", "date"])
        .reset_index(drop=True)
    )
    facility_long.to_csv("hotel_facility_monthly_long.csv", index=False, encoding="utf-8-sig")
    print("✔ hotel_facility_monthly_long.csv を出力しました")
except Exception as e:
    print(f"エラー: 施設数データ処理中に問題が発生しました: {e}")
    facility_long = None

# データマージと指標構築

In [ ]:
print("\n--- III. データマージと指標構築 ---")

if df_master is not None and facility_long is not None:
    try:
        hotel_master = df_master.copy()

        # ========= date を月初に揃える =========
        for df in (hotel_master, facility_long):
            df["date"] = pd.to_datetime(df["date"])
            df["date"] = df["date"].dt.to_period("M").dt.to_timestamp()

        # ========= マージ（pref_name × date） =========
        keys = ["pref_name", "date"]

        hotel_with_supply = hotel_master.merge(
            facility_long[keys + ["facility_total"]],
            on=keys,
            how="left",
            validate="m:1"
        )

        hotel_with_supply.to_csv("hotel_master_with_facility.csv", index=False, encoding="utf-8-sig")
        print("✔ hotel_master_with_facility.csv 保存完了")

        # --- 需給指標の構築 ---
        df = hotel_with_supply.copy()

        # year / month が無ければ date から作る
        if "year" not in df.columns:
            df["year"] = df["date"].dt.year
        if "month" not in df.columns:
            df["month"] = df["date"].dt.month

        # 元データ側の数値列を一旦すべて数値型に揃える
        base_numeric_cols = ["total_guest", "foreigner_guest", "occ", "facility_total"]
        for col in base_numeric_cols:
            df[col] = pd.to_numeric(df[col], errors="coerce")

        # 1. 外国人比率 inbound_share
        df["inbound_share"] = df["foreigner_guest"] / df["total_guest"]

        # 2. 施設数の標準化 facility_std
        scaler = StandardScaler()
        # NaNは0で埋めて標準化計算。結果はFloatで上書き。
        df["facility_std"] = scaler.fit_transform(df["facility_total"].fillna(0).to_frame())

        # 3. 需給バランス指数 balance_index = occ / facility_std
        df["balance_index"] = df["occ"] / df["facility_std"].replace(0, np.nan) # ゼロ除算回避

        # 4. 需給比 demand_supply_ratio = total_guest / facility_total
        df["demand_supply_ratio"] = df["total_guest"] / df["facility_total"].replace(0, np.nan) # ゼロ除算回避

        # 5. 2019年同月比指数（occ / demand_supply_ratio）
        base_2019 = (
            df[df["year"] == 2019]
            .loc[:, ["pref_name", "month", "occ", "demand_supply_ratio"]]
            .rename(
                columns={
                    "occ": "occ_2019",
                    "demand_supply_ratio": "dsr_2019",
                }
            )
        )

        df = df.merge(base_2019, on=["pref_name", "month"], how="left")
        df["occ_index_2019"] = df["occ"] / df["occ_2019"].replace(0, np.nan)
        df["dsr_index_2019"] = df["demand_supply_ratio"] / df["dsr_2019"].replace(0, np.nan)

        # 追加した指標列をすべて数値型に揃える
        new_numeric_cols = [
            "inbound_share", "facility_std", "balance_index", "demand_supply_ratio",
            "occ_2019", "dsr_2019", "occ_index_2019", "dsr_index_2019",
        ]
        for col in new_numeric_cols:
            df[col] = pd.to_numeric(df[col], errors="coerce")

        # 年次データを作成（年間平均・合計）
        df_yearly = df.groupby(['pref_name', 'pref_code', 'year']).agg(
            occ=('occ', 'mean'),
            demand_supply_ratio=('demand_supply_ratio', 'mean'),
            inbound_share=('inbound_share', 'mean'),
            total_guest=('total_guest', 'sum'),
            japanese_guest=('japanese_guest', 'sum'),
            foreigner_guest=('foreigner_guest', 'sum'),
            facility_total=('facility_total', 'mean')
        ).reset_index()

        # 年次データにも2019年比指数を適用
        base_2019_yearly = df_yearly[df_yearly['year'] == 2019].rename(
            columns={'occ': 'occ_2019_yearly', 'demand_supply_ratio': 'dsr_2019_yearly'}
        )[['pref_name', 'occ_2019_yearly', 'dsr_2019_yearly']]

        df_yearly = df_yearly.merge(base_2019_yearly, on='pref_name', how='left')
        df_yearly['occ_index_2019_yearly'] = df_yearly['occ'] / df_yearly['occ_2019_yearly'].replace(0, np.nan)
        df_yearly['dsr_index_2019_yearly'] = df_yearly['demand_supply_ratio'] / df_yearly['dsr_2019_yearly'].replace(0, np.nan)

        df.to_csv("hotel_master_with_facility_with_index.csv", index=False, encoding="utf-8-sig")
        df_yearly.to_csv("hotel_master_yearly_with_index.csv", index=False, encoding="utf-8-sig")

        print("✔ hotel_master_with_facility_with_index.csv 保存完了")
        print("✔ hotel_master_yearly_with_index.csv (年次) 保存完了")

    except Exception as e:
        print(f"エラー: マージまたは指標構築中に問題が発生しました: {e}")
else:
    print("データ処理が不完全なため、マージ・指標構築をスキップしました。")
    # 仮のファイル名を設定して、以降の分析コードがエラーにならないようにする
    FILE_NAME_MONTHLY = 'hotel_master_with_facility_with_index.csv'
    FILE_NAME_YEARLY = 'hotel_master_yearly_with_index.csv'


# 可視化と構造分析

In [ ]:
print("\n--- IV. 可視化と構造分析 ---")

# --- データの読み込みと前処理 ---

FILE_NAME_MONTHLY = 'hotel_master_with_facility_with_index.csv'
FILE_NAME_YEARLY = 'hotel_master_yearly_with_index.csv'
BASE_YEAR = 2019 # 比較基準年

try:
    df_monthly = pd.read_csv(FILE_NAME_MONTHLY, parse_dates=['date'])
    df_yearly = pd.read_csv(FILE_NAME_YEARLY)
except FileNotFoundError:
    print(f"エラー: 分析に必要なファイルが見つかりません。")
    exit()

# データ型の変換とクリーニング (df_monthly)
df_monthly['yyyymm'] = df_monthly['date'].dt.strftime('%Y-%m')

numeric_cols = ['total_guest', 'japanese_guest', 'foreigner_guest', 'occ', 'facility_total', 'inbound_share']
for col in numeric_cols:
    df_monthly[col] = pd.to_numeric(df_monthly[col], errors='coerce')

if "year" not in df_monthly.columns:
    df_monthly["year"] = df_monthly["date"].dt.year
if "month" not in df_monthly.columns:
    df_monthly["month"] = df_monthly["date"].dt.month

# ここで df_monthly を二つに分ける
# 1. トレンドプロット用: 稼働率(occ)があればよい。facility_totalがNaNでも残す。
df_monthly_for_trend = df_monthly.dropna(subset=['occ']).copy()

# 2. 構造分析用: facility_totalを含め、すべての必須列が揃っているもののみを使用
required_cols_for_structural = ['occ', 'foreigner_guest', 'japanese_guest', 'inbound_share', 'facility_total']
df_monthly_for_structural = df_monthly.dropna(subset=required_cols_for_structural).copy()


# --- 分析対象年とメジャーの決定 ---
# LATEST_ANALYSIS_YEARは構造分析ができる最新年 (facility_totalが欠損していない年)
LATEST_ANALYSIS_YEAR = df_monthly_for_structural['year'].max()
print(f"分析統一基準年 (構造分析): {LATEST_ANALYSIS_YEAR}年")


# 3-1. 稼働率の変動係数 (リスク指標 - X軸) -- df_monthly_for_structural を使用
risk_metrics = df_monthly_for_structural.groupby('pref_name')['occ'].agg(
    std=('std'),
    mean=('mean')
).reset_index()
risk_metrics['変動係数'] = risk_metrics['std'] / risk_metrics['mean']
risk_metrics = risk_metrics.drop(columns=['std', 'mean'])


# 3-2. 稼働率施設効率比 (OCC/Facility Ratio) - Y軸として稼働率を使用する
# df_latest_ratio はここでは直接使用せず、df_latest_baseのocc_base_avgを使う

latest_efficiency_data = df_yearly[df_yearly['year'] == LATEST_ANALYSIS_YEAR].groupby('pref_name').agg(
    Y軸_稼働率=('occ', 'mean'), # Y軸を年間平均稼働率に直接変更
    最新年_施設数=('facility_total', 'mean'), # ホバー情報用
    最新年_延べ宿泊者数=('total_guest', 'sum') # バブルサイズ用 (需要総量)
).reset_index()


# 3-3. 構造的特性の計算 (最新年 LATEST_ANALYSIS_YEAR のデータを使用) -- df_yearly (これも2025はNaN) を使用
# df_yearly は cmQZ6IwS8jsD で作られているので、これはそのまま
df_latest_base = df_yearly[df_yearly['year'] == LATEST_ANALYSIS_YEAR].groupby('pref_name').agg(
    total_guest_latest=('total_guest', 'sum'),
    foreigner_guest_latest=('foreigner_guest', 'sum'),
    occ_latest_avg=('occ', 'mean')
).reset_index()
df_latest_base.rename(columns={
    'total_guest_latest': 'total_guest_base',
    'foreigner_guest_latest': 'foreigner_guest_base',
    'occ_latest_avg': 'occ_base_avg'
}, inplace=True)
df_latest_base['インバウンド依存度_Base'] = df_latest_base['foreigner_guest_base'] / df_latest_base['total_guest_base']


# 3-4. 最新年インバウンド依存度の計算 (色付け/ホバー情報用)
latest_inbound_data = df_latest_base.rename(columns={'インバウンド依存度_Base': '最新年_インバウンド依存度'})
latest_inbound_data = latest_inbound_data[['pref_name', '最新年_インバウンド依存度']]


# 3-5. 稼働率回復指数 (ランキング用) -- df_yearly を使用
recovery_rank_df = df_yearly[df_yearly['year'] == LATEST_ANALYSIS_YEAR].groupby('pref_name')['occ_index_2019_yearly'].mean().reset_index()
recovery_rank_df.rename(columns={'occ_index_2019_yearly': '最新年_稼働率回復指数'}, inplace=True)


# 全ての分析指標を結合 (analysis_df: セグメンテーションマップの元データ)
analysis_df = latest_efficiency_data.merge(risk_metrics, on='pref_name', how='inner')
analysis_df = analysis_df.merge(df_latest_base[['pref_name', 'インバウンド依存度_Base', 'occ_base_avg']], on='pref_name', how='left')
analysis_df = analysis_df.merge(latest_inbound_data[['pref_name', '最新年_インバウンド依存度']], on='pref_name', how='left')
analysis_df = analysis_df.merge(recovery_rank_df, on='pref_name', how='left')

# ホバーラベルの調整
analysis_df.rename(columns={'インバウンド依存度_Base': f'インバウンド依存度_{LATEST_ANALYSIS_YEAR}年',
                           'occ_base_avg': f'occ_avg_{LATEST_ANALYSIS_YEAR}年'}, inplace=True)


# --- 4. 可視化: Page 1 - 回復度ランキング ---

print("\n--- 1. 稼働率 2019年比 回復度ランキング ---")

ranking_df = analysis_df.sort_values(by='最新年_稼働率回復指数', ascending=True).dropna(subset=['最新年_稼働率回復指数'])

fig1 = px.bar(ranking_df,
              x='最新年_稼働率回復指数',
              y='pref_name',
              orientation='h',
              title=f'{LATEST_ANALYSIS_YEAR}年 年間平均稼働率 2019年比 回復指数ランキング',
              labels={'最新年_稼働率回復指数': '稼働率回復指数 (2019年=1.0)', 'pref_name': '都道府県'},
              color='最新年_稼働率回復指数',
              color_continuous_scale=px.colors.sequential.Viridis,
              height=900)

fig1.add_vline(x=1.0, line_dash="dash", line_color="red", annotation_text="回復基準 (1.0)", annotation_position="top right")
fig1.update_layout(yaxis={'categoryorder':'total ascending'})
fig1.show()


# --- 5. 可視化: Page 2 - 月次稼働率トレンド ---

print("\n--- 2. 全国平均 月次稼働率の回復軌跡 ---")

# プロット対象年: 2019年 (基準), 2024年, 2025年
plot_years = {BASE_YEAR, 2024, 2025}
df_trend = df_monthly_for_trend[df_monthly_for_trend['year'].isin(plot_years)]

df_monthly_avg = df_trend.groupby(['year', 'month', 'yyyymm'])['occ'].mean().reset_index()
df_monthly_avg['year_str'] = df_monthly_avg['year'].astype(str)

# DEBUG: 実際にプロット対象となる年を確認
print(f"DEBUG: 実際にデータに含まれる年 (df_monthly_avg): {df_monthly_avg['year_str'].unique().tolist()}")

# 色の設定:
color_map = {}
color_map[str(BASE_YEAR)] = '#d62728'       # 2019年: 赤 (基準)
color_map['2024'] = '#1f77b4'              # 2024年: 青
color_map['2025'] = '#2ca02c'              # 2025年: 緑 (強調1)

# 実際にデータに含まれる年のみをフィルタリングしてマップに適用
filtered_color_map = {year: color for year, color in color_map.items() if year in df_monthly_avg['year_str'].unique()}

# 凡例の順序を決定 (2019, 2024, 2025の順)
plot_order_preference = [str(BASE_YEAR), '2024', '2025']
if str(LATEST_ANALYSIS_YEAR) not in plot_order_preference:
    plot_order_preference.append(str(LATEST_ANALYSIS_YEAR))

sorted_color_map = {year: color_map[year] for year in plot_order_preference if year in filtered_color_map}
plot_years_list = [int(y) for y in sorted_color_map.keys()]
title_years = ', '.join([f'{y}年' for y in plot_years_list])


fig2 = px.line(df_monthly_avg,
               x='month',
               y='occ',
               color='year_str',
               title=f'全国平均 月次稼働率の回復軌跡 ({title_years})',
               labels={'occ': '平均稼働率 (%)', 'month': '月', 'year_str': '年'},
               color_discrete_map=sorted_color_map,
               category_orders={"year_str": [str(y) for y in plot_years_list]}, # 凡例の順番を確保
               markers=True,
               height=500)

# 各線の幅を調整
for trace in fig2.data:
    year = trace.name
    # 基準年、2024年、最新年 の線を太くする
    if year in filtered_color_map:
        trace.line.width = 3.0
    else:
        trace.line.width = 1.0

fig2.update_layout(xaxis = dict(tickmode = 'linear', tick0 = 1, dtick = 1))
fig2.show()


# --- 6. 可視化: Page 3 - 構造的セグメンテーション (散布図) ---

print("\n--- 3. 地域構造セグメンテーションマップ (規模調整済み効率と変動リスク) ---")
print("バブルサイズは「延べ宿泊者数（需要総量）」です。左上（高効率）でバブルが小さい県は「安定したミニ市場」を意味します。")

x_mean = analysis_df['変動係数'].mean()
y_mean = analysis_df[f'occ_avg_{LATEST_ANALYSIS_YEAR}年'].mean() # Y軸の平均値を稼働率に変更

def assign_segment(row):
    risk = '高変動（ハイリスク）' if row['変動係数'] >= x_mean else '低変動（安定）'
    # Y軸: 年間平均稼働率
    efficiency = '高稼働（効率的）' if row[f'occ_avg_{LATEST_ANALYSIS_YEAR}年'] >= y_mean else '標準稼働（改善余地あり）'
    return f"{risk} / {efficiency}"

analysis_df['セグメント'] = analysis_df.apply(assign_segment, axis=1)

fig3 = px.scatter(analysis_df,
                  x='変動係数',
                  y=f'occ_avg_{LATEST_ANALYSIS_YEAR}年', # Y軸を年間平均稼働率に直接変更
                  size='最新年_施設数', # バブルサイズを「施設数」（供給規模）に変更
                  color='最新年_インバウンド依存度',
                  hover_data={f'インバウンド依存度_{LATEST_ANALYSIS_YEAR}年': ':.1%',
                              '最新年_インバウンド依存度': ':.1%',
                              f'occ_avg_{LATEST_ANALYSIS_YEAR}年': ':.1f', # ホバー情報も稼働率に調整
                              '最新年_稼働率回復指数': ':.2f',
                              '最新年_施設数': ':.0f', # ホバー情報として施設数を維持
                              'セグメント': True},
                  hover_name='pref_name',
                  title=f'稼働率と変動リスクによる地域セグメンテーション ({LATEST_ANALYSIS_YEAR}年データ基準)', # タイトルも調整
                  labels={'変動係数': '月次稼働率 変動係数 (リスク)',
                          f'occ_avg_{LATEST_ANALYSIS_YEAR}年': f'{LATEST_ANALYSIS_YEAR}年 年間平均稼働率 (%)', # Y軸ラベルを稼働率に変更
                          '最新年_施設数': f'{LATEST_ANALYSIS_YEAR}年 施設数 (供給規模)', # バブルサイズラベルを更新
                          '最新年_インバウンド依存度': f'{LATEST_ANALYSIS_YEAR}年 インバウンド依存度 (現在の特性)'},
                  color_continuous_scale=px.colors.sequential.Turbo,
                  height=600)

fig3.add_hline(y=y_mean, line_dash="dash", line_color="blue",
               annotation_text=f"稼働率 平均: {y_mean:.1f}", annotation_position="bottom right") # アノテーションも稼働率に調整
fig3.add_vline(x=x_mean, line_dash="dash", line_color="red",
               annotation_text=f"変動係数 平均: {x_mean:.2f}", annotation_position="top left")

# バブルサイズの説明注釈を「施設数」に変更
fig3.add_annotation(
    x=analysis_df['変動係数'].max(),
    y=analysis_df[f'occ_avg_{LATEST_ANALYSIS_YEAR}年'].max(), # Y座標を稼働率に調整
    xref="x", yref="y",
    text=f"バブルサイズ：{LATEST_ANALYSIS_YEAR}年 施設数（供給規模）",
    showarrow=False,
    font=dict(size=10, color="darkslategray"),
    xanchor="right", yanchor="top",
    xshift=-10, yshift=-10,
    bgcolor="rgba(255, 255, 255, 0.7)",
    bordercolor="#cccccc",
    borderwidth=1,
    borderpad=4
)
# ----------------------------------------

# 都道府県ラベルをプロットに追加
fig3.add_trace(go.Scatter(
    x=analysis_df['変動係数'],
    y=analysis_df[f'occ_avg_{LATEST_ANALYSIS_YEAR}年'], # Y軸を稼働率に調整
    mode='text',
    text=analysis_df['pref_name'],
    textposition="middle center",
    textfont=dict(
        size=8,
        color="black"
    ),
    showlegend=False,
    hoverinfo='none'
))

fig3.show()

# --- 7. 戦略提言 (分析結果の言語化) ---

print("\n--- 4. 戦略提言とアクションプラン (実務補正版 - 効率性・変動リスク・供給規模に基づき) ---") # 見出し変更
print("バブルサイズ（施設数＝供給規模）を考慮に入れ、各地域特性に合わせた戦略を提言します。") # 説明変更
print("-" * 50)

segments = analysis_df.groupby('セグメント')['pref_name'].apply(list).to_dict()

# 1. 低変動（安定） / 高稼働（効率的）
if '低変動（安定） / 高稼働（効率的）' in segments:
    area = ", ".join(segments['低変動（安定） / 高稼働（効率的）'])
    print(f"【競争優位・安定成長型】 ({area})")
    print("  - **構造**: 高い稼働率を安定的に達成。**バブルサイズ（施設数）が大きい場合は、多くの施設が高い稼働率を誇る、市場として魅力的な地域**。バブルサイズが小さい場合は「安定したニッチ市場」と読み替える。インバウンド依存度については、平均より高いか低いかを確認し、戦略を調整する。") # コメント変更
    print("  - **提言**: 市場の安定性を維持しつつ、**ADRの最大化**と、ブランド力強化による**市場シェアの独占**を推進。過疎県は地域外需要を取り込む高付加価値化。")
    print("-" * 50)

# 2. 高変動（ハイリスク） / 高稼働（効率的）
if '高変動（ハイリスク） / 高稼働（効率的）' in segments:
    area = ", ".join(segments['高変動（ハイリスク） / 高稼働（効率的）'])
    print(f"【集中需要・分散リスク型】 ({area})")
    print("  - **構造**: 高い稼働率を達成しているが、需要が特定の時期/イベントに集中し不安定。**バブルサイズ（施設数）が大きい場合は、多くの施設が季節変動のリスクに晒されている**ことを意味する。インバウンド需要への依存度が高い地域もこの傾向が強い。") # コメント変更
    print("  - **提言**: **閑散期を埋めるための徹底的な需要平準化**（MICE、長期ビジネス誘致）を推進。需要が集中するハイシーズンは、リスクを恐れず高値戦略を徹底。")
    print("-" * 50)

# 3. 低変動（安定） / 標準稼働（改善余地あり）
if '低変動（安定） / 標準稼働（改善余地あり）' in segments:
    area = ", ".join(segments['低変動（安定） / 標準稼働（改善余地あり）'])
    print(f"【安定供給・コスト最適化型】 ({area})")
    print("  - **構造**: 変動リスクは低いが、稼働率が平均水準以下で安定している。大都市周辺や中核都市で多く見られ、需要を取りこぼしている可能性がある。**バブルサイズ（施設数）が大きい場合は、多くの施設が低稼働で安定している可能性**があり、市場全体の供給過多を示唆する。") # コメント変更
    print("  - **提言**: **徹底したオペレーションの効率化**と固定費削減。競争力の低い施設は**ブランドチェンジやリポジショニング**によるテコ入れ。県内主要都市（例: 横浜、神戸）の粒度分析を推奨。")
    print("-" * 50)

# 4. 高変動（ハイリスク） / 標準稼働（改善余地あり）
if '高変動（ハイリスク） / 標準稼働（改善余地あり）' in segments:
    area = ", ".join(segments['高変動（ハイリスク） / 標準稼働（改善余地あり）'])
    print(f"【構造的リスク・需給変革型】 ({area})")
    print("  - **構造**: 稼働率が平均水準以下かつ変動も激しいため、投資リスクが最も高い。**バブルサイズ（施設数）が大きい場合は、市場全体が構造的な問題を抱えており、危機的状況**にあることを意味する。インバウンド依存度が高く、市場外のショックに弱い。") # コメント変更
    print("  - **提言**: **単なる販促ではなく、供給構造と依存度の抜本的変革**が必須。競争力の低い資産の**用途転換**（例: 宿泊から長期賃貸へ）による供給調整。国内安定需要（レジデンス需要やワーケーションなど）の徹底的な開拓。")
    print("-" * 50)


# ==============================================================================
# --- 8. 回復指数による構造分析 ---
# ==============================================================================

print("\n--- 5. 回復指数（最新年 vs 2019年）の傾向分析 ---")
print(f"稼働率回復指数に基づき、構造的特性（{LATEST_ANALYSIS_YEAR}年効率比、施設数）を分析します。")

# --- 分析設定 ---
N = 10

# 必要な列を持つデータフレームを用意し、NaNを除外
analysis_for_ranking = analysis_df.dropna(subset=['最新年_稼働率回復指数', f'occ_avg_{LATEST_ANALYSIS_YEAR}年', '最新年_施設数']).copy() # Y軸指標の列名変更
analysis_for_ranking.sort_values(by='最新年_稼働率回復指数', ascending=False, inplace=True)

# --- グループ分け ---
high_recovery = analysis_for_ranking.head(N)
low_recovery = analysis_for_ranking.tail(N)

# --- 分析結果の計算 ---
analysis_data = {
    'グループ': ['回復指数 高 (上位10)', '回復指数 低 (下位10)'],
    f'平均_{LATEST_ANALYSIS_YEAR}年稼働率 (%)': [
        f"{high_recovery[f'occ_avg_{LATEST_ANALYSIS_YEAR}年'].mean():.1f}",
        f"{low_recovery[f'occ_avg_{LATEST_ANALYSIS_YEAR}年'].mean():.1f}"
    ],
    '平均_稼働率': [f"{high_recovery[f'occ_avg_{LATEST_ANALYSIS_YEAR}年'].mean():.1f}", f"{low_recovery[f'occ_avg_{LATEST_ANALYSIS_YEAR}年'].mean():.1f}"], # Y軸指標の列名変更
    '平均_変動係数': [f"{high_recovery['変動係数'].mean():.2f}", f"{low_recovery['変動係数'].mean():.2f}"],
    '平均_施設数 (件)': [
        f"{analysis_df[analysis_df['pref_name'].isin(high_recovery['pref_name'])]['最新年_施設数'].mean():.0f}",
        f"{analysis_df[analysis_df['pref_name'].isin(low_recovery['pref_name'])]['最新年_施設数'].mean():.0f}"
    ],
    '都道府県 (上位)': [", ".join(high_recovery['pref_name'].tolist()), ""],
    '都道府県 (下位)': ["", ", ".join(low_recovery['pref_name'].tolist())]
}

analysis_summary = pd.DataFrame(analysis_data).set_index('グループ')
print(analysis_summary.to_markdown(numalign="left", stralign="left"))

print("-" * 70)

# --- 傾向の言語化 ---

# 1. 回復指数と稼働率の関係 (効率比から稼働率に変更)
high_occ_mean = high_recovery[f'occ_avg_{LATEST_ANALYSIS_YEAR}年'].mean() # 列名変更
low_occ_mean = low_recovery[f'occ_avg_{LATEST_ANALYSIS_YEAR}年'].mean()   # 列名変更

if high_occ_mean > low_occ_mean:
    tendency_1 = f"**傾向①：回復指数が高い地域は、平均稼働率が高い傾向**にあります。回復指数が高いグループの平均稼働率（{high_occ_mean:.1f}%）は、低いグループの平均（{low_occ_mean:.1f}%）を大きく上回っています。これは、**回復の鍵が、高い稼働率を維持する市場の魅力や運営効率**にあることを示唆します。"
else:
    tendency_1 = f"**傾向①：回復指数が高い地域と平均稼働率の関係に明確な相関は見られません。**回復指数が高いグループの平均稼働率（{high_occ_mean:.1f}%）は、低いグループの平均（{low_occ_mean:.1f}%）を下回るか同程度です。これは、**回復は絶対的な需要の増加によってもたらされ、必ずしも高い稼働率とは結びついていない**可能性を示唆します。"


# 2. 回復指数と変動係数の関係 (リスク)
high_risk_mean = high_recovery['変動係数'].mean()
low_risk_mean = low_recovery['変動係数'].mean()

# 小数点以下の実際の値に基づいて比較し、丸めによる表示上の差異を説明に含める
if high_risk_mean > low_risk_mean:
    tendency_2 = f"**傾向②：回復指数が高い地域は、変動リスクが高い傾向にありますが、その差はわずかです。**回復指数が高いグループの平均変動係数（{high_risk_mean:.4f}）は、低いグループの平均変動係数（{low_risk_mean:.4f}）をわずかに上回っています。表示上は同じ『{high_risk_mean:.2f}』と丸められるこの小さな差から、回復が特定の需要に集中し、**季節性などのリスクが高まっていると断定することは難しい**ですが、変動の可能性には留意が必要です。"
elif high_risk_mean < low_risk_mean:
    tendency_2 = f"**傾向②：回復指数が高い地域は、変動リスクが低い傾向**にあります。回復指数が高いグループの平均変動係数（{high_risk_mean:.4f}）は、低いグループの平均変動係数（{low_risk_mean:.4f}）よりも低く、**安定した需要回復**を示唆しています。変動係数とは月次稼働率の変動度合いを示すため、変動の少ない安定的な地域ほど回復が早い、という構造が見えます。"
else:
    tendency_2 = f"**傾向②：回復指数と変動リスクの関係に明確な相関は見られません。**平均変動係数は高・低グループ間で同程度（約{high_risk_mean:.4f}）です。"


# 3. 回復指数と施設数 (市場規模/供給量) の関係
try:
    # analysis_summaryから文字列として施設数を取得し、数値に変換
    high_facility_mean = float(analysis_summary.loc['回復指数 高 (上位10)', '平均_施設数 (件)'].replace(',', '').replace(' ', ''))
    low_facility_mean = float(analysis_summary.loc['回復指数 低 (下位10)', '平均_施設数 (件)'].replace(',', '').replace(' ', ''))
except ValueError:
    tendency_3 = "**傾向③：施設数による分析は、データ型エラーのため処理をスキップしました。**"
    high_facility_mean = low_facility_mean = None

if high_facility_mean is not None and low_facility_mean is not None:
    if high_facility_mean < low_facility_mean:
        tendency_3 = f"**傾向③：回復指数が高い地域の平均施設数は、回復が遅い地域よりも小さい**傾向（{high_facility_mean:.0f}件 vs {low_facility_mean:.0f}件）にあります。これは、**市場規模が小さく、供給過多になりにくいニッチな市場**の方が、国内レジャーなどの回復を先行して捉えやすかった可能性を示唆しています。**施設数が少ない地域ほど、供給過多のリスクが低く、需要変動に柔軟に対応しやすい**と考えられます。"
    elif high_facility_mean > low_facility_mean:
        tendency_3 = f"**傾向③：回復指数が高い地域の平均施設数は、回復が遅い地域よりも大きい**傾向（{high_facility_mean:.0f}件 vs {low_facility_mean:.0f}件）にあります。これは、**東京や大阪のような巨大都市圏（インバウンド依存度が高い）**の施設数が多く、それらの都市が他を凌駕するペースで回復していることを示唆しています。ただし、このグループには『高変動 / 標準稼働』な地域が多く含まれている場合、**施設数が多いことによる競争激化や供給過多のリスク**に注意が必要です。"
    else:
        tendency_3 = f"**傾向③：回復指数が高い地域と低い地域の間で、施設規模（平均施設数）に大きな差は見られません。**平均施設数はそれぞれ約{high_facility_mean:.0f}件と同水準です。"


print("\n--- 傾向の結論 ---")
print(tendency_1)
print(tendency_2)
print(tendency_3)
print("\n結論: 回復度合いは、単なる絶対需要だけでなく、市場の供給規模（施設数）に対する需要集中度（効率性）と、需要の季節的な安定性（変動リスク）によって構造的に決定されています。")


# --- 9. 実行環境へのフィードバック ---
print("\n--- 実行結果の確認 ---")
# 最終マスターデータ行数は df_monthly_for_trend の行数を確認
if df_monthly_for_trend is not None:
    print(f"最終マスターデータ行数 (トレンド分析用): {df_monthly_for_trend.shape[0]}")
    print(f"最終マスターデータ最新日付 (トレンド分析用): {df_monthly_for_trend['date'].max()}")
else:
    print("データ処理が完了していません。Google Driveのマウント状況とファイルパスを確認してください。")